In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import csv
import pandas as pd

In [8]:
# Cargar datos del excel

df = pd.read_csv('https://drive.google.com/uc?id=1HE95YCW-Ypg971nJTOwXiM3E0R7hkghG', encoding='cp1252', delimiter=';')

df.head()
df

,FECHA_CORTE,NRO_REPORTE,FECHA_ACC,HORA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,LATITUD,LONGITUD,PROGRESIVA,CODIGO_VIA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20211130,001.01-001.2020 SC,20200101,23:40:00,CHOQUE,0,0,-15.456740,-74.866607,521.00,PE-1S,AREQUIPA,CARAVELI,LOMAS
1,20211130,002.01-002.2020 CC,20200101,07:45:00,DESPISTE,0,1,-10.824119,-76.199915,103.10,PE-3N,PASCO,PASCO,VICCO
2,20211130,004.01-004.2020 CC,20200101,06:20:00,CHOQUE,0,2,-9.531989,-77.531930,16.00,NaN,ANCASH,HUARAZ,HUARAZ
3,20211130,036.01-036.2020 CC,20200101,02:30:00,DESPISTE,0,2,-5.903333,-78.161667,263.80,PE-5N,AMAZONAS,UTCUBAMBA,JAMALCA
4,20211130,005.01-005.2020 CC,20200102,00:40:00,CHOQUE,2,6,-10.201361,-78.050778,275.00,PE-1N,ANCASH,HUARMEY,HUARMEY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3827,20211130,7121002195 CC,20211130,20:15:00,CHOQUE,0,1,-16.567602,-72.626707,863.31,PE-1S,AREQUIPA,CAMANA,SAMUEL PASTOR
3828,20211130,7121002197 CC,20211130,00:00:00,VOLCADURA,1,1,-13.527485,-73.182984,65.00,AP-100,APURIMAC,ANDAHUAYLAS,KISHUARA
3829,20211130,7121002198 SC,20211130,22:40:00,DESPISTE,0,0,-13.127046,-76.393172,146.80,PE-1S,LIMA,CAÑETE,SAN VICENTE DE CAÑETE
3830,20211130,7121002199 CC,20211130,18:00:00,ACCIDENTE,1,0,-6.420472,-77.872722,283.30,PE-08B,AMAZONAS,CHACHAPOYAS,MAGDALENA


In [9]:
# DROP_COLUMNS = ['FECHA_CORTE', 'NRO_REPORTE', 'FECHA_ACC', 'HORA_ACC', 'PROGRESIVA','CODIGO_VIA']
# # Mostrar las primeras filas del DataFrame
# df.head()

# Corregir la lista de columnas a eliminar
DROP_COLUMNS = ['FECHA_CORTE', 'NRO_REPORTE','LATITUD','LONGITUD','PROGRESIVA','DEPARTAMENTO','PROVINCIA','DISTRITO']

# Eliminar las columnas especificadas
df.drop(DROP_COLUMNS, axis=1, inplace=True)

# Mostrar las primeras filas del DataFrame
df

,FECHA_ACC,HORA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,CODIGO_VIA
0,20200101,23:40:00,CHOQUE,0,0,PE-1S
1,20200101,07:45:00,DESPISTE,0,1,PE-3N
2,20200101,06:20:00,CHOQUE,0,2,NaN
3,20200101,02:30:00,DESPISTE,0,2,PE-5N
4,20200102,00:40:00,CHOQUE,2,6,PE-1N
...,...,...,...,...,...,...
3827,20211130,20:15:00,CHOQUE,0,1,PE-1S
3828,20211130,00:00:00,VOLCADURA,1,1,AP-100
3829,20211130,22:40:00,DESPISTE,0,0,PE-1S
3830,20211130,18:00:00,ACCIDENTE,1,0,PE-08B


In [10]:
# convertir datos de hora a texto por tunos (mañana, tarde y noche)
turnos = {
    'mañana': 1,
    'tarde': 2,
    'noche': 3
}
def convertir_hora_a_turnos(hora):
    if hora >= pd.to_datetime('20:00:00').time() or hora < pd.to_datetime('06:00:00').time():
        return turnos['noche']
    elif hora >= pd.to_datetime('06:00:00').time() and hora < pd.to_datetime('12:00:00').time():
        return turnos['mañana']
    else:
        return turnos['tarde']

# Aplicar la función a la columna de horas
df['TURNOS_ACC'] = df['HORA_ACC'].apply(lambda x: convertir_hora_a_turnos(pd.to_datetime(x).time()))

# Guardar el resultado en un nuevo archivo csv
df.to_csv('/content/drive/MyDrive/accidentes_transformado.csv', index=False)


In [11]:
# mostrar dataframe cuando aparece la columna "HORA_ACC" y "TURNO_ACC"
df

,FECHA_ACC,HORA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,CODIGO_VIA,TURNOS_ACC
0,20200101,23:40:00,CHOQUE,0,0,PE-1S,3
1,20200101,07:45:00,DESPISTE,0,1,PE-3N,1
2,20200101,06:20:00,CHOQUE,0,2,NaN,1
3,20200101,02:30:00,DESPISTE,0,2,PE-5N,3
4,20200102,00:40:00,CHOQUE,2,6,PE-1N,3
...,...,...,...,...,...,...,...
3827,20211130,20:15:00,CHOQUE,0,1,PE-1S,3
3828,20211130,00:00:00,VOLCADURA,1,1,AP-100,3
3829,20211130,22:40:00,DESPISTE,0,0,PE-1S,3
3830,20211130,18:00:00,ACCIDENTE,1,0,PE-08B,2


In [12]:
# Cargar datos
df = pd.read_csv('/content/drive/MyDrive/accidentes_transformado.csv')

# Corregir la lista de columnas a eliminar
DROP_COLUMNS = ['HORA_ACC']

# Eliminar las columnas especificadas
df.drop(DROP_COLUMNS, axis=1, inplace=True)
df.head()
df

# Persistir la eliminación de columnas
df.to_csv('/content/drive/MyDrive/accidentes_transformado.csv', index=False)

In [13]:
# Eliminar filas con CODIGO_VIA inexistente
df.dropna(subset='CODIGO_VIA', inplace=True)

# Persistir eliminación
df.to_csv('/content/drive/MyDrive/accidentes_transformado.csv', index=False)

df

,FECHA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,CODIGO_VIA,TURNOS_ACC
0,20200101,CHOQUE,0,0,PE-1S,3
1,20200101,DESPISTE,0,1,PE-3N,1
3,20200101,DESPISTE,0,2,PE-5N,3
4,20200102,CHOQUE,2,6,PE-1N,3
6,20200102,DESPISTE,2,16,PE-1SJ,2
...,...,...,...,...,...,...
3827,20211130,CHOQUE,0,1,PE-1S,3
3828,20211130,VOLCADURA,1,1,AP-100,3
3829,20211130,DESPISTE,0,0,PE-1S,3
3830,20211130,ACCIDENTE,1,0,PE-08B,2


In [14]:
# Extraer códigos de via unicos
codigos_via_unicos = set(df['CODIGO_VIA'])

# Construir diccionario de códigos de via
codigos_via = { cod: i + 1 for i, cod in enumerate(codigos_via_unicos) }
pd.Series(codigos_via)

PA-104       1
PE-18B       2
CU-125       3
CU-130       4
AN-106       5
          ... 
PE-34K     175
PE-3S      176
CU-1279    177
CA-106     178
PU-100     179
Length: 179, dtype: int64

In [15]:
# Transformar CODIGO_VIA a un valor numérico
df['CODIGO_VIA'] = df['CODIGO_VIA'].apply(lambda cod: codigos_via[cod])

# Persistir transformación
df.to_csv('/content/drive/MyDrive/accidentes_transformado.csv', index=False)

df

,FECHA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,CODIGO_VIA,TURNOS_ACC
0,20200101,CHOQUE,0,0,148,3
1,20200101,DESPISTE,0,1,27,1
3,20200101,DESPISTE,0,2,115,3
4,20200102,CHOQUE,2,6,25,3
6,20200102,DESPISTE,2,16,164,2
...,...,...,...,...,...,...
3827,20211130,CHOQUE,0,1,148,3
3828,20211130,VOLCADURA,1,1,106,3
3829,20211130,DESPISTE,0,0,148,3
3830,20211130,ACCIDENTE,1,0,126,2


In [16]:
# extraer modalidades
modalidades_unicas = set(df['MODALIDAD_ACC'])

# Construir diccionario de modalidades
modalidades = { mod: i for i, mod in enumerate(modalidades_unicas) }
pd.Series(modalidades)

ESPECIAL     0
CHOQUE       1
EMBISTE      2
ACCIDENTE    3
ATROPELLO    4
INCENDIO     5
VOLCADURA    6
DESPISTE     7
dtype: int64

In [17]:
# Transformar MODALIDAD_ACC a un valor numérico
df['MODALIDAD_ACC'] = df['MODALIDAD_ACC'].apply(lambda mod: modalidades[mod])

# Persistir transformación
df.to_csv('/content/drive/MyDrive/accidentes_transformado.csv', index=False)

df

,FECHA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,CODIGO_VIA,TURNOS_ACC
0,20200101,1,0,0,148,3
1,20200101,7,0,1,27,1
3,20200101,7,0,2,115,3
4,20200102,1,2,6,25,3
6,20200102,7,2,16,164,2
...,...,...,...,...,...,...
3827,20211130,1,0,1,148,3
3828,20211130,6,1,1,106,3
3829,20211130,7,0,0,148,3
3830,20211130,3,1,0,126,2


In [18]:
# Convertir la columna FECHA_ACC a formato datetime

df['FECHA_ACC'] = pd.to_datetime(df['FECHA_ACC'], format='%Y%m%d')

# Modificar el formato datetime para mostrar solo año y mes (YYYYMM)

df['FECHA_ACC'] = df['FECHA_ACC'].dt.strftime('%Y%m')

# Persistir transformación
df.to_csv('/content/drive/MyDrive/accidentes_transformado.csv', index=False)

df

,FECHA_ACC,MODALIDAD_ACC,CANT_FALLECIDOS,CANT_HERIDOS,CODIGO_VIA,TURNOS_ACC
0,202001,1,0,0,148,3
1,202001,7,0,1,27,1
3,202001,7,0,2,115,3
4,202001,1,2,6,25,3
6,202001,7,2,16,164,2
...,...,...,...,...,...,...
3827,202111,1,0,1,148,3
3828,202111,6,1,1,106,3
3829,202111,7,0,0,148,3
3830,202111,3,1,0,126,2
